In [197]:
import pandas as pd
import numpy as np
import os
import shutil
import string

In [226]:
df = pd.read_excel(r'C:/EU-Study/Pandan/JT_project/Лудза.xls')

In [179]:
# Создаем df для записи в него папки происхождения изображения, его оригинального имени и имени, усеченного до идентификационного номера

df_old_path = pd.DataFrame(columns=['folder',
                                    'im_name',
                                    'id']) 

# Копируем все изображения из одной папки в другую, сохраняя их метаданные в таблицу

old_path = 'C:/EU-Study/Pandan/JT_project/old/'
new_path = 'C:/EU-Study/Pandan/JT_project/new/'

folders = os.listdir(path=old_path)

for f in folders:
    im_list = os.listdir(path=old_path+f)

    for im in im_list:
        
        # Многие изображения снимаются по нескольку раз и в названиях им делают постфиксы "_1"|"_2" или "a"|"b", но
        # в таблицах с переводами они значаться без них. соответственно для мёрджа нужно их убрать:
        im_id = im.split('.')[0].split('_')[0]
        im_id = im_id.rstrip(string.ascii_letters + string.whitespace)        

        src_dir = old_path+str(f)+'/'+str(im)
        dst_dir = new_path+str(im)
        shutil.copy(src_dir,dst_dir)

        df_old_path = df_old_path.append({'folder': f,
                                          'im_name': im.split('.')[0],
                                          'id': im_id}, ignore_index=True)  



In [185]:
# Создаем общую таблицу для таблиц переводов текстов с надгробий

df_text = pd.DataFrame(columns=['id', 'jewish_text', 'rus_transl'])

fileDir = r"C:/EU-Study/Pandan/JT_project/"
file_list = [_ for _ in os.listdir(fileDir) if _.endswith(".xlsx")]
file_list.extend([_ for _ in os.listdir(fileDir) if _.endswith(".xls")])
file_list.remove('Оргеев_координаты.xlsx')


for f in file_list:
    
    df_temp = pd.read_excel(r'C:/EU-Study/Pandan/JT_project/'+f)
    df_temp.columns = ['id', 'jewish_text', 'rus_transl']
    
    df_text = df_text.append(df_temp)

In [260]:
# Мерджим две части

df = pd.merge(df_text, df_old_path, how='outer', indicator=True)
df.replace({'-': None},inplace =True)
df['rus_transl'] = df['rus_transl'].replace('\\n', ' ', regex=True)

df

,id,jewish_text,rus_transl,folder,im_name,_merge
0,ORG001,פנ\n איש הר‘\n יעקב בהר‘\n משה מזרחי\n שנפטר י...,"Здесь похоронен мужчина, р. Яаков, сын р. М...",Оргеев,ORG001,both
1,ORG001,פנ\n איש הר‘\n יעקב בהר‘\n משה מזרחי\n שנפטר י...,"Здесь похоронен мужчина, р. Яаков, сын р. М...",Оргеев,ORG001_1,both
2,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002,both
3,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002_1,both
4,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002_3,both
...,...,...,...,...,...,...
6104,SDB088,NaN,NaN,Стародуб,SDB088_2,right_only
6105,SDB089,NaN,NaN,Стародуб,SDB089,right_only
6106,SDB089,NaN,NaN,Стародуб,SDB089a,right_only
6107,SDB098,NaN,NaN,Стародуб,SDB098,right_only


In [262]:
# Посмотрим на результат:

df._merge.value_counts()

# Слишком много left_only. Ручная проверка показывает, что это в основном те изображения, 
# у которых в самом названии есть буквы (а мы их убирали выше). В таком случае сделаем отдельный мёрдж по left_only

both          5319
left_only      457
right_only     333
Name: _merge, dtype: int64

In [263]:
df_with_letter_in_id = df[df._merge == "left_only"].drop(['folder', 'im_name', '_merge'], 1)

df_with_letter_in_id = pd.merge(df_with_letter_in_id, df_old_path.drop('id', 1), how='left', left_on='id', right_on='im_name', indicator=True)
df_with_letter_in_id

,id,jewish_text,rus_transl,folder,im_name,_merge
0,ORG074A,פנ\n ... שלמה\n יעקב [שנ]\n חו ניסן שנת\n תקפט...,"Здесь похоронен … Шломо Яаков, [который скон...",NaN,NaN,left_only
1,ORG074B,פנ\n איש חשב הר\n שמאל בה יצחק\n שנ ךב בטבת\n ...,"Здесь похоронен почтенный человек, р. Шмуэль...",NaN,NaN,left_only
2,ORG150,…\n ...לפ\n תנצבה,… По малому [летоисчислению]. Да будет душа ...,NaN,NaN,left_only
3,ORG170,NaN,NaN,NaN,NaN,left_only
4,ORG252A,פנ\n הילד ישראל\n בהר דניאל שנ‘\n יום טז‘ בחו’...,"Здесь покоится ребенок Исраэль, сын р. Даниэ...",NaN,NaN,left_only
...,...,...,...,...,...,...
452,LDZ5158b,פנ\nהרב\nמופ חוב יא מו\nמשה נתן בר דן\nנ כִד ט...,"Здесь похоронен раввин, наш выдающийся, усердн...",Лудза,LDZ5158b,both
453,LDZ2054a,פנ\nהאשה\nגיטא בת\nאברהם\nחייט\nנפ ב כסלו\nתרסו,"Здесь похоронена женщина Гита, дочь Авраама, Х...",Лудза,LDZ2054a,both
454,LDZ2054b,פנ\nחיה שרה\nבר יעקב\nקאראקאווסקע\nנפט ח סיון\...,"Здесь похоронена Хая Сара, дочь р. Яакова, Кар...",Лудза,LDZ2054b,both
455,LDZ2070a,None,None,Лудза,LDZ2070a,both


In [264]:
# Проверим:
df_with_letter_in_id._merge.value_counts()

# Ручная проверка показывает, что оставшиеся 112 - это те, название которых в таблице не соответствует названию 
# файла, либо просто такого файла нет, тут уже ничего не поделать.

left_only     345
both          112
right_only      0
Name: _merge, dtype: int64

In [269]:
# Соединим два блока:
df_final = pd.concat([df[df._merge != "left_only"], df_with_letter_in_id])
df_final


,id,jewish_text,rus_transl,folder,im_name,_merge
0,ORG001,פנ\n איש הר‘\n יעקב בהר‘\n משה מזרחי\n שנפטר י...,"Здесь похоронен мужчина, р. Яаков, сын р. М...",Оргеев,ORG001,both
1,ORG001,פנ\n איש הר‘\n יעקב בהר‘\n משה מזרחי\n שנפטר י...,"Здесь похоронен мужчина, р. Яаков, сын р. М...",Оргеев,ORG001_1,both
2,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002,both
3,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002_1,both
4,ORG002,פנ\n איש חשוב\n הרר יצחק\n אייזק בהרר\n יצחק ה...,Здесь похоронен почтенный человек р. Ицхак ...,Оргеев,ORG002_3,both
...,...,...,...,...,...,...
452,LDZ5158b,פנ\nהרב\nמופ חוב יא מו\nמשה נתן בר דן\nנ כִד ט...,"Здесь похоронен раввин, наш выдающийся, усердн...",Лудза,LDZ5158b,both
453,LDZ2054a,פנ\nהאשה\nגיטא בת\nאברהם\nחייט\nנפ ב כסלו\nתרסו,"Здесь похоронена женщина Гита, дочь Авраама, Х...",Лудза,LDZ2054a,both
454,LDZ2054b,פנ\nחיה שרה\nבר יעקב\nקאראקאווסקע\nנפט ח סיון\...,"Здесь похоронена Хая Сара, дочь р. Яакова, Кар...",Лудза,LDZ2054b,both
455,LDZ2070a,None,None,Лудза,LDZ2070a,both


In [270]:
df_final._merge.value_counts()

both          5431
left_only      345
right_only     333
Name: _merge, dtype: int64

In [ ]:
# Я не стал мёрджить с координатами, т.к. там есть проблемы в несовпадающих id, которые нужно исправлять + их не так много (чуть больше 800). Если необходимо, могу сделать 

In [271]:
df.to_csv('C:/EU-Study/Pandan/JT_project/sefer_unite_df.csv', index=False)